In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
warnings.filterwarnings("ignore")

# Load the engagement file 
engagement_list = []
import os
for dirname, _, filenames in os.walk('/kaggle/input/learnplatform-covid19-impact-on-digital-learning'):
    for filename in filenames:
        file = os.path.join(dirname, filename)
        if file[-4:] == '.csv' and filename not in ['products_info.csv','districts_info.csv']:
            district_id = int(filename.split('.')[0])
            df = pd.read_csv(file)
            df['district_id'] = district_id
            engagement_list.append(df)

engagement_data = pd.concat(engagement_list)
# Lodd the product information file 
products_info = pd.read_csv('/kaggle/input/learnplatform-covid19-impact-on-digital-learning/products_info.csv')
# Load the districts information file 
districts_info = pd.read_csv('/kaggle/input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv')

print(f'There are {engagement_data.shape[0]} rows and {engagement_data.shape[1]} columns in the engagement dataset.\n')
print(f'There are {products_info.shape[0]} rows and {products_info.shape[1]} columns in the products dataset.\n')
print(f'There are {districts_info.shape[0]} rows and {districts_info.shape[1]} columns in the districts info dataset.')

# Preprocessing the products_info dataframe #

In [ ]:
# Delete products that there Functions are unknown 
products_info = products_info[products_info['Primary Essential Function'].notna()]
# Delete rows that do not mention the product name 
products_info = products_info[products_info['Product Name'].notna()]

# Rename the product id column so it can be joined later with 
# the engagement dataframe 
products_info.rename(columns={'LP ID':'lp_id'}, inplace=True)

# TBD later on 
products_info['Product Category'] = np.NaN

# Split the Primary Essential Function to Primary Function 
# and Secondary Function 
primary_functions = []
secondary_functions = []
for row in products_info['Primary Essential Function']:
    primary_functions.append(row.split('-')[0])
    secondary_functions.append(row.split('-')[1])
products_info['Primary Function'] = primary_functions
products_info['Secondary Function'] = secondary_functions 

# One-hot encode the Educational sectors that the product is used
product_sectors = products_info['Sector(s)'].str.get_dummies(sep="; ")
products_info = pd.concat([products_info,product_sectors], axis = 1)

# Delete the Primary Essential Function column since its not needed now  
del products_info['Primary Essential Function']
del products_info['Sector(s)']
products_info

# Preprocessing the districts_info dataframe 

In [ ]:
# Delete rows where the district_id is not available 
districts_info = districts_info[districts_info['state'].notna()]

columns = ['pct_black/hispanic','county_connections_ratio','pct_free/reduced','pp_total_raw']

def data_wrangling(df,column): 
    '''
    Calculates the midpoint of variables where their values 
    are given as a range. 
    '''
    ls = []
    x = 'avg_' + str(column) 
    df[x] = np.NaN
    for row in df[column]:
        if type(row) == float:
            # Append NA so the length is the same and 
            # replace later with the column avg 
            ls.append(np.NaN) 
        else:
            ls.append((float(row.split('[')[1].split(',')[0]) + float(row.split('[')[1].split(',')[1])) / 2)
    ls = [round(x,2) for x in ls]
    df[x] = ls  
                   
    del df[column]
    return df

for column in columns:
    districts_info = data_wrangling(districts_info,column)

# Replace NAs with the average values per state and locale 
districts_info['avg_pp_total_raw'] = districts_info.groupby(['state','locale'])['avg_pp_total_raw'].\
                                     fillna(int(districts_info['avg_pp_total_raw'].mean()))

districts_info['avg_pct_free/reduced'] = districts_info.groupby(['state','locale'])['avg_pct_free/reduced'].\
                                         fillna(round(districts_info['avg_pct_free/reduced'].mean(),2))

districts_info['avg_county_connections_ratio'] = districts_info.groupby(['state','locale'])['avg_county_connections_ratio'].\
                                                 fillna(districts_info['avg_county_connections_ratio'].mean())

# Add another column for the % of people that are not black/hispanic 
districts_info['avg_pct_other'] = 1 - districts_info['avg_pct_black/hispanic']
districts_info.head()

In [ ]:
# Inner join the engagement dataframe with the products_info dataframe 
# Engagement data that do not contain state/product will be dropped 
engagement_data =  engagement_data.merge(products_info, on='lp_id', how='inner')

# Inner join the engagement dataframe with the ditricts_info dataframe 
engagement_data = engagement_data.merge(districts_info, on = 'district_id', how = 'inner')

In [ ]:
# Summary statistics for the engagment dataframe after the joins 
engagement_data.describe()

# EDA - DEMOGRAPHICS AND USER BEHAVIOR #

In [ ]:
# Black / hispanic population per state 
from matplotlib import pyplot as plt
avg_pct = districts_info.groupby(['state'])['avg_pct_black/hispanic'].mean().sort_values(ascending = True)
avg_pct.T.plot(kind = "barh", figsize=(22, 13))
plt.suptitle("Black / Hispanic population percentage per (school district) State", fontsize = 31)

In [ ]:
# Summary Statistics per state 
states_summary = pd.DataFrame()
states_summary['observations'] = engagement_data.groupby(['state'])['state'].count().sort_values(ascending = False)
states_summary['start_date'] = engagement_data.groupby('state')['time'].min()
states_summary['end_date'] = engagement_data.groupby('state')['time'].max()
states_summary['locales'] = engagement_data.groupby(['state'])['locale'].nunique()
states_summary['products_used'] = engagement_data.groupby(['state'])['lp_id'].nunique()
states_summary['product_usage_ratio'] = round((states_summary['products_used'] / len(products_info)) * 100,2)
states_summary['product_categories'] = np.NaN
states_summary

### Observations:
* North Dakota has the least amount of data out of the featured states (check how many educational instituions present)
* Most states have data for the full year, although many of them were not on lockdown but for a few months (investigate)
* Almost all states have used to some capacity all the available tools for digitial (distance) learning.  
* 
*

### TO DO:
* Check land capacity and population per state
* Break down per state -> locale
*

# EDA - PRODCUTS SECTION #

In [ ]:
# Top 20 products used 
prodcut_popularity = engagement_data.groupby(['Product Name'])['Product Name'].count()\
                                .sort_values(ascending = False)

prodcut_popularity[:20].T.plot(kind = "bar", figsize=(22, 12))
plt.suptitle("Top 20 most used products", fontsize = 31)

In [ ]:
# Top 20 products with the highetst engagement index 
prodcut_engagement_index = engagement_data.groupby(['Product Name'])['engagement_index'].mean()\
                                .sort_values(ascending = False)

prodcut_engagement_index[:20].T.plot(kind = "bar", figsize=(22, 12))
plt.suptitle("Engagemnt Index for the top 20 products", fontsize = 31)

# EDA - ENGAGEMENT SECTION #

In [ ]:
import matplotlib.pyplot as plt
state_avg_engagement_idx = engagement_data.groupby(['state'])['engagement_index'].mean().sort_values(ascending = False)
state_avg_engagement_idx.T.plot(kind = "bar", figsize=(22, 13))
plt.suptitle("Average Engagement Index per State", fontsize = 31)

In [ ]:
locale_avg_engagement_idx = engagement_data.groupby(['locale'])['engagement_index'].mean().sort_values(ascending = False)
locale_avg_engagement_idx.T.plot(kind = "bar", figsize=(20, 8))
plt.suptitle("Average Engagement Index per Locale", fontsize = 30)

### It seems that on average the rural areas have a higer engagement index. This could be becuase people living away from the cities were using digital channels for learning from before. Overall, its worth investigating how the engagement index per locale changes as a function of time, since some locales might adjusted to online-learning faster than others.  

In [ ]:
state_locale_avg_engagement_idx = engagement_data.groupby(['state','locale'])['engagement_index'].mean().sort_values(ascending = True)
state_locale_avg_engagement_idx.T.plot(kind = "barh", figsize=(23, 25))
plt.suptitle("Engagement Index per State and Locale", fontsize = 32)

In [ ]:
# Engagement index as a function of time 
engagement_data['time'] = pd.to_datetime(engagement_data['time'])
engagement_data.sort_values(by='time') # This now sorts in date order

daily_avg_engagement_idx = engagement_data.groupby(['time'])['engagement_index'].mean().sort_values()
daily_avg_engagement_idx.T.plot(kind = "line", figsize=(15, 7))

## Observations:
* At first the engagemnent was lower (could be either because students / institution did not quickly utilize online learning or deifferent locales shut down at different periods)
* Around mid-February, where most places were on lockdown, the engagement index doubled. 
* Overall, there is high activity over the weekdays, with extreme spikes during the weekends (students spend less time studying over the weekends, regardles of the Covid pandemic)
* Near summer, the engagement index started to diminish, since most educational institutions were closed. 


In [ ]:
mask = (engagement_data['time'] > '2020-01-01') & (engagement_data['time'] <= '2020-02-01')
mask = engagement_data.loc[mask]
mask.groupby(['time'])['engagement_index'].mean().sort_values().T.plot(kind = "line", figsize=(15, 7))

### Zooming in the first month, we can indeed see that the activity occurs in the weekdays. Additionally, the dataset comprises of time-series data. A seasonal ARIMA model can be used to model the future engagement index both on a per state and locale level.

In [ ]:
black_hispanic_engagement = engagement_data.groupby(['avg_pct_black/hispanic'])['engagement_index'].mean().sort_values(ascending = False)
black_hispanic_engagement.T.plot(kind = 'bar',stacked = True, figsize=(18, 7))

In [ ]:
avg_state_pct_BlackHispanic = engagement_data['avg_pct_black/hispanic'].mean() 
avg_BlackHispanic_engagement_index = engagement_data.groupby(['avg_pct_black/hispanic'])['engagement_index'].mean().mean() 

print(f'The average percentage of black/hispanic ethnicities per state is {round(avg_state_pct_BlackHispanic,2) * 100}% with \
an average engagement index of {round(avg_BlackHispanic_engagement_index,2)}')

## Please 👍 and let me know that you want to see more, so i know to update this notebook with further data / analysis and modelling.   